In [7]:
import pandas as pd

In [9]:
data_person = pd.read_csv("CASC_Constant.csv")
print(data_person.head())

   CustomerId ActivationDate   Age     Sex  SubscribedEmail  SubscribedPush
0     2728183     2015-01-01  21.0  Female            False            True
1     2728198     2015-01-01  21.0  Female             True            True
2     2728306     2015-01-01  21.0  Female             True            True
3     2728178     2015-01-01  22.0    Male             True            True
4     2728322     2015-01-01  22.0    Male             True            True


In [16]:
data_transact = pd.read_csv("casc-resto-small.csv")
print(data_transact.head())

   CustomerID  Restaurant      RKDate                    RegionName  \
0     2898197         391  2017-07-16  Москва и Московская область    
1     2903215          43  2015-04-07  Москва и Московская область    
2     2748887          43  2015-05-22  Москва и Московская область    
3     2862077          46  2015-03-05  Москва и Московская область    
4     2862077          46  2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
0  TGI FRIDAYS      NON ALCOHOL         2          2                   2.0  
1     IL Патио      NON ALCOHOL         1          5                   5.0  
2     IL Патио      NON ALCOHOL         1          5                   5.0  
3     IL Патио      NON ALCOHOL         1          5                   5.0  
4     IL Патио      NON ALCOHOL         1          5                   5.0  


In [19]:
print(data_transact.describe())
print(data_transact.info())

         CustomerID   Restaurant     Quantity    SummBasic  \
count  1.000000e+03  1000.000000  1000.000000  1000.000000   
mean   2.801380e+06   243.463000     1.101000     8.944000   
std    5.994973e+04   258.801799     0.418299     2.039859   
min    2.728549e+06    40.000000     1.000000     2.000000   
25%    2.751840e+06    54.000000     1.000000    10.000000   
50%    2.771859e+06   116.000000     1.000000    10.000000   
75%    2.862317e+06   392.000000     1.000000    10.000000   
max    2.913068e+06   934.000000     5.000000    10.000000   

       SummAfterPointsUsage  
count           1000.000000  
mean               8.099960  
std                2.451883  
min                2.000000  
25%                5.020000  
50%               10.000000  
75%               10.000000  
max               10.000000  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
CustomerID              1000 non-null int64
Restaurant            

In [30]:
data_transact[(data_transact.SummAfterPointsUsage != data_transact.SummBasic) & (data_transact.Quantity != 1)].head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
212,2758212,721,2015-06-17,Москва и Московская область,"IL Патио,Планета Суши",FOOD,5,9,5.07
222,2750233,40,2016-11-18,Москва и Московская область,Американский Бар и Гриль,FOOD,2,10,5.00
226,2730340,43,2015-01-03,Москва и Московская область,IL Патио,NON ALCOHOL,2,10,9.86
236,2888593,43,2015-05-24,Москва и Московская область,IL Патио,NON ALCOHOL,2,10,5.10
246,2733081,43,2015-08-16,Москва и Московская область,IL Патио,FOOD,2,10,6.26


In [20]:
data_transact[data_transact.SummAfterPointsUsage > data_transact.SummBasic]

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage


In [28]:
data_transact.RKDate = pd.to_datetime(data_transact.RKDate)
print(data_transact.head())
type(data_transact.RKDate[0])

   CustomerID  Restaurant     RKDate                    RegionName  \
0     2898197         391 2017-07-16  Москва и Московская область    
1     2903215          43 2015-04-07  Москва и Московская область    
2     2748887          43 2015-05-22  Москва и Московская область    
3     2862077          46 2015-03-05  Москва и Московская область    
4     2862077          46 2015-03-11  Москва и Московская область    

   BrandsNames DishCategoryName  Quantity  SummBasic  SummAfterPointsUsage  
0  TGI FRIDAYS      NON ALCOHOL         2          2                   2.0  
1     IL Патио      NON ALCOHOL         1          5                   5.0  
2     IL Патио      NON ALCOHOL         1          5                   5.0  
3     IL Патио      NON ALCOHOL         1          5                   5.0  
4     IL Патио      NON ALCOHOL         1          5                   5.0  


pandas._libs.tslibs.timestamps.Timestamp

In [32]:
data_transact = data_transact.sort_values("RKDate")
data_transact.tail()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
209,2767598,450,2017-11-22,Москва и Московская область,Мама Раша,FOOD,1,9,7.00
773,2768840,183,2017-11-29,Новосибирск,IL Патио,NON ALCOHOL,1,10,10.00
717,2821271,129,2017-12-28,Москва и Московская область,IL Патио,FOOD,1,10,10.00
803,2750656,295,2017-12-29,Москва и Московская область,Шикари,FOOD,1,10,6.87
849,2748926,390,2017-12-30,Москва и Московская область,IL Патио,FOOD,1,10,10.00


In [37]:
data_transact[data_transact.RKDate == pd.to_datetime('2017-07-01')].head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage


In [104]:
data_transact[data_transact.RKDate >= pd.to_datetime('2017-07-01')].head()

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
271,2910374,43,2017-07-06,Москва и Московская область,IL Патио,FOOD,1,10,10.00
683,2831289,93,2017-07-15,Москва и Московская область,IL Патио,FOOD,1,10,4.99
0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2,2.00
814,2913068,298,2017-08-02,Москва и Московская область,Шикари,FOOD,1,10,10.00
417,2751226,49,2017-08-03,Москва и Московская область,IL Патио,FOOD,1,10,8.92


In [91]:
rkdate = data_transact.RKDate[(data_transact.RKDate >= pd.to_datetime('2017-07-01')) & (data_transact.RKDate <= pd.to_datetime('2017-12-31'))]

463

In [135]:
data_transact.groupby(["CustomerID", rkdate]).count()['Restaurant']

CustomerID  RKDate    
2732734     2017-10-22    1
2743625     2017-11-07    1
2747453     2017-11-11    1
2748926     2017-12-30    1
2749468     2017-08-05    1
2750656     2017-12-29    1
2751226     2017-08-03    2
2756140     2017-08-28    1
2767598     2017-11-22    1
2768840     2017-11-29    1
2798123     2017-11-19    2
2814898     2017-09-19    1
2820372     2017-11-06    1
2821271     2017-12-28    1
2831289     2017-07-15    1
2838143     2017-11-20    1
2884152     2017-08-18    1
2898197     2017-07-16    1
2910374     2017-07-06    1
2912991     2017-11-18    1
2913068     2017-08-02    1
Name: Restaurant, dtype: int64

CustomerID  RKDate    
2732734     2017-10-22    1
Name: Restaurant, dtype: int64